In [1]:
!pip install datasets python-Levenshtein --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd
from Levenshtein import ratio

In [3]:
ds = load_dataset('linxinyuan/cola')
ds_train = pd.DataFrame(ds["train"])
ds_test = pd.DataFrame(ds["test"])

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cola downloaded and prepared to /root/.cache/huggingface/datasets/linxinyuan___cola/default/0.0.0/0871d55203d4de46ef1815400998ed8f219236694f0d03786bde849741f04cd4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
def calc_dist(row):
    return ratio(row["sentence1"], row["sentence2"])

def substitude(row):
    s1 = set(row["sentence1"].replace(".", "").split())
    s2 = set(row["sentence2"].replace(".", "").split())
    if (s1 - s2) and (s2 - s1):
        return 1
    return 0

def prepare_ds(ds):
    dataset = pd.concat([
        ds.iloc[:-1].rename(columns={"text": "sentence1", "label": "label1"}),
        ds.iloc[1:].rename(columns={"text": "sentence2", "label": "label2"}).reset_index(drop=True)
    ], axis=1, join="inner")
    dataset["dist"] = dataset.apply(calc_dist, axis=1)
    dataset["match"] = dataset.apply(lambda row: 1 if row["label1"]==row["label2"] else 0, axis=1)
    dataset["sub"] = dataset.apply(substitude, axis=1)
    return dataset

In [5]:
result_ds = pd.concat([prepare_ds(ds_train), prepare_ds(ds_test)], ignore_index=True)

In [6]:
result_ds[(result_ds["dist"]>0.7) & (result_ds["sub"] == 1) & (result_ds["match"] == 0)]

,sentence1,label1,sentence2,label2,dist,match,sub
42,They made him to exhaustion.,0,They made him into a monster.,1,0.736842,0,1
54,The witch went into the forest by vanishing.,0,The witch went into the forest and thereby van...,1,0.854167,0,1
60,This building got taller and taller.,1,This building is taller and taller.,0,0.929577,0,1
77,Bill cried himself to sleep.,1,Bill cried Sue to sleep.,0,0.846154,0,1
95,The ball wriggled itself loose.,0,Bill wriggled himself loose.,1,0.813559,0,1
...,...,...,...,...,...,...,...
8900,the book of poems and from Blackwell takes a v...,0,the one with a red cover takes a very long tim...,1,0.725806,0,1
8912,Heidi thinks that Andy to eat salmon flavored ...,0,Heidi thinks that Andy should eat salmon flavo...,1,0.949153,0,1
8917,"If I am a rich man, I'd buy a diamond ring.",0,"If he were a rich man, he'd buy a diamond ring.",1,0.866667,0,1
8981,I want to try and buy some whiskey.,1,She goes and buying some whiskey.,0,0.705882,0,1


In [ ]:
result_ds.to_csv("cola.csv")